In [3]:
import numpy as np
import scipy.constants as sciconsts
import scipy.fftpack as fft
import scipy.interpolate as sci_interpolate
import scipy.integrate as sci_integrate
import time
import sys

def chirp_mass(m1,m2):
    return np.power(m1*m2,0.6)/(np.power(m1+m2,0.2))
def solvem(mc,smr):
    a=mc/np.power(smr,0.6)
    b=smr*np.power(a,2.0)
    m1=(a+np.sqrt(np.power(a,2.0)-4.0*b))/2.0
    m2=(a-np.sqrt(np.power(a,2.0)-4.0*b))/2.0
    return np.array((m1,m2))
def sym_mass_ratio(m1,m2):
    return m1*m2/(np.power(m1+m2,2.0))

'''my Tmodel(considering gas (SI))'''
def k_1cal(m_1,m_2):
    return 64.0*np.power(sciconsts.G,3.0)*m_1*m_2*(m_1+m_2)/5.0/np.power(sciconsts.c,5.0)
def t_0cal(m_1,m_2,t_gas,a_0):
    return t_gas*0.25*np.log(k_1cal(m_1,m_2)+np.power(a_0,4.0)/t_gas)
def acalgas(t,k_1,t_gas,t_0):
    return np.power((np.exp(4.0*(t_0-t)/t_gas)-k_1)*t_gas,0.25)
def freq_tgas(t,m_1,m_2,t_gas,t_0):
    k_1=k_1cal(m_1,m_2)
    a_ft=acalgas(t,k_1,t_gas,t_0)
    return 1.0/sciconsts.pi*np.sqrt((m_1+m_2)*sciconsts.G)*np.power(a_ft,-1.5)
def phi_tgas(t,m_1,m_2,t_gas,t_0):
    return sci_integrate.quad(lambda x:2*sciconsts.pi*freq_tgas(x,m_1,m_2,t_gas,t_0),0,t)[0]
def hgas(t,m_1,m_2,t_gas,t_0,phi):
    k_1=k_1cal(m_1,m_2)
    return m_1*m_2/acalgas(t,k_1,t_gas,t_0)*np.cos(phi)/1e70
'''my Tmode not considering gas'''
def acalvac(t,k_1,t_c):
    return np.power(4*k_1*(t_c-t),0.25)
def freq_tvac(t,m_1,m_2,t_c):
    avt=acalvac(t,k_1cal(m_1,m_2),t_c)
    return np.sqrt(sciconsts.G*(m_1+m_2))/sciconsts.pi*np.power(avt,-1.5)
def phi_tvac(t,m_1,m_2,t_c):
    cm2=chirp_mass(m_1,m_2)
    return -2*np.power(np.power(sciconsts.c,3.0)/5.0/cm2/sciconsts.G*(t_c-t),5.0/8.0)
def hvac(t,m_1,m_2,t_c,phi_c):
    return  m_1*m_2/acalvac(t,k_1cal(m_1,m_2),t_c)*np.cos(phi_tvac(t,m_1,m_2,t_c)+phi_c)/1e70
#phi_c[0,2pi]
def f_jugg(t_c,m_1,m_2):
    return (freq_tvac(0,m_1,m_2,t_c),freq_tvac(4*years,m_1,m_2,t_c))
def t_fvac(f,m_c,t_c):
    sci1=np.power(sciconsts.c,3.0)/5.0/m_c/sciconsts.G
    return t_c-np.power(8.0*sciconsts.pi*f/5.0,-8.0/3.0)*np.power(sci1,5.0/3.0)
def tlimit(m_c,fi,ff):
    sci1=np.power(sciconsts.c,3.0)/5.0/m_c/sciconsts.G
    a=np.array((np.power(8.0*sciconsts.pi*fi/5.0,-8.0/3.0)*np.power(sci1,5.0/3.0),4*years+np.power(8.0*sciconsts.pi*ff/5.0,-8.0/3.0)*np.power(sci1,5.0/3.0)))
    return [a.min(),a.max()]
def tcn_decide(mc,fi,ff,acu):
    return ((tlimit(mc,fi,ff)[1]-tlimit(mc,figas,ffgas)[0])/tlimit(mc,fi,ff)[1])//acu+1
def noden_cal(mc,phinum,tcn_racu,fi,ff):
    mcn=mc.size
    i=0
    sum1=0
    while i<mcn:
        sum1=sum1+tcn_decide(mc[i],fi,ff,tcn_racu)
        i=i+1
    return sum1*phinum


def pow2(a):
    b=1
    i=True
    while i:
        b=2*b
        if b>a:
            i=False
    return b
'''lisa noise curve in rpsd unit N2A5:laserpower 2w,40cm telescope,armlength 5e6 km'''
def S_gal_N2A5(f):
    if f>=1.0e-5 and f<1.0e-3:
        return np.power(f,-2.3)*np.power(10,-44.62)*20.0/3.0
    if f>=1.0e-3 and f<np.power(10,-2.7):
        return np.power(f,-4.4)*np.power(10,-50.92)*20.0/3.0
    if f>=np.power(10,-2.7) and f<np.power(10,-2.4):
        return np.power(f,-8.8)*np.power(10,-62.8)*20.0/3.0
    if f>=np.power(10,-2.4) and f<=0.01:
        return np.power(f,-20.0)*np.power(10,-89.68)*20.0/3.0
    if f>0.01 or f<1.0e-5:
        return 0
def S_n_lisa(f):
    m1=5.0e9
    m2=sciconsts.c*0.41/m1/2.0
    return 20.0/3.0*(1+np.power(f/m2,2.0))*(4.0*(9.0e-30/np.power(2*sciconsts.pi*f,4.0)*(1+1.0e-4/f))+2.96e-23+2.65e-23)/np.power(m1,2.0)+S_gal_N2A5(f)

#consts setting
m_sun=1.9891e30
years=365*24*3600.0
mpc=3.261*sciconsts.light_year
t_scale=4*years
#gas cal
m_1g=10*m_sun
m_2g=10*m_sun
k_1g=k_1cal(m_1g,m_2g)
t_gas=1000.0*years
a_0=3.0e8
t_0g=t_0cal(m_1g,m_2g,t_gas,a_0)
#(t_scale*0.01*4,N/0.01/t_scale,t_scale) 
figas=freq_tgas(0,m_1g,m_2g,t_gas,t_0g)
ffgas=freq_tgas(t_scale,m_1g,m_2g,t_gas,t_0g)
Ng=10000000
t=np.linspace(0,t_scale,num=Ng)
#tdomain interp1d
tphi=np.linspace(0,t_scale,10000)
i=0
workphi=np.zeros(10000)
while i<10000:
    workphi[i]=phi_tgas(tphi[i],m_1g,m_2g,t_gas,t_0g)
    i=i+1
phimo=sci_interpolate.interp1d(tphi,workphi)
#tdomain point cal
Tg=t_scale/Ng
hfgas=hgas(t,m_1g,m_2g,t_gas,t_0g,phimo(t))
#fft
xs=np.linspace(0,1/2.0/Tg,num=Ng//2)
hfgas1=fft.fft(hfgas)
hfgas_abs=abs(hfgas1)
hfgas_angle=np.arctan(hfgas1.imag/hfgas1.real)[0:Ng//2]
hfgas2=2.0/Ng*hfgas_abs[0:Ng//2]
hs=sci_interpolate.interp1d(xs,hfgas2)
anglegas=sci_interpolate.interp1d(xs,hfgas_angle)

A=np.sqrt(sci_integrate.quad(lambda x:4*np.power(hs(x),2.0)/S_n_lisa(x),figas,ffgas))
#parameters changes
mcnum=10000
phinum=10
acu=0.0001
mcs=np.linspace(38*m_sun,39*m_sun,mcnum)
phis=np.linspace(0,2*sciconsts.pi,phinum)
data=np.zeros((mcnum,2))
Tesi=noden_cal(mcs,phinum,0.0001,figas,ffgas)/3600
print 'cal start,esiT=',Tesi,'h'
sys.stdout.flush()
mcn=0
tb=time.clock()
while mcn<mcnum:
    ffmax=0.0
    phimaxn=0
    tcmaxn=0
    ffmax=0.0
    errmax=0.0
    m1v=solvem(mcs[mcn],0.25)[0]
    m2v=solvem(mcs[mcn],0.25)[1]
    tcl=tlimit(mcs[mcn],figas,ffgas)
    tcnum=tcn_decide(mcs[mcn],figas,ffgas,acu)
    tcl_sp=np.linspace(tcl[0],tcl[1],tcnum)
    tcn=0
    maxprosess=1.0
    while tcn<tcnum:
        phin=0
        while phin<phinum:
            ti=time.clock()
            fivac=freq_tvac(0,m1v,m2v,tcl_sp[tcn])
            ffvac=freq_tvac(4*years,m1v,m2v,tcl_sp[tcn])
            Ng=int(t_scale*ffvac*4)
            Ng=pow2(Ng)
            T=t_scale/Ng
            xs=np.linspace(0,1/2.0/T,num=Ng//2)
            t=np.linspace(0,t_scale,num=Ng)
            hvt=hvac(t,m1v,m2v,tcl_sp[tcn],phis[phin])
            hfvac1=fft.fft(hvt)
            hfvac_abs=abs(hfvac1)
            hfvac_angle=np.arctan(hfvac1.imag/hfvac1.real)[0:Ng//2]
            anglevac=sci_interpolate.interp1d(xs,hfvac_angle)
            hfvac2=2.0/Ng*hfvac_abs[0:Ng//2]
            hvacs=sci_interpolate.interp1d(xs,hfvac2)
            d=(np.array((fivac,figas)).max(),np.array((ffvac,ffgas)).min())
            B=np.sqrt(sci_integrate.quad(lambda x:4*np.power(hvacs(x),2.0)/S_n_lisa(x),fivac,ffvac))
            AB=sci_integrate.quad(lambda x:4*hvacs(x)*hs(x)/S_n_lisa(x)*np.cos(anglegas(x)-anglevac(x)),d[0],d[1])
            ff=AB[0]/A[0]/B[0]
            err=abs(1/A[0]/B[0]*AB[1])+abs(1/A[0]/A[0]/B[0]*A[1])+abs(1/A[0]/B[0]/B[0]*B[1])
            tf=time.clock()
            if ff>ffmax:
                ffmax=ff
                errmax=err
                phimaxn=phin
                tcmaxn=tcn
            #print mcs[mcn]/m_sun,ff
            #sys.stdout.flush()
            phin=phin+1
            #maxprosess=maxprosess+1
        tcn=tcn+1
    data[mcn,0]=mcs[mcn]/m_sun
    data[mcn,1]=ffmax
    data[mcn,2]=errmax
    print mcs[mcn]/m_sun,ffmax
    sys.stdout.flush()
    mcn=mcn+1
np.savetxt('gwdata.txt',data)
te=time.clock()
print 'cal completed!',(te-tb)/3600,'h'
sys.stdout.flush()



cal start,esiT= 279.683333333 h


/Users/shyn/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:154: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.


38.0 0.6
38.0 1.2
38.0 1.8
38.0 2.4
38.0 3.0
38.0 3.6
38.0 4.2
38.0 4.8
38.0 5.4
38.0 6.0
38.0 6.6
38.0 7.2
38.0 7.8
38.0 8.4
38.0 9.0
38.0 9.6
38.0 10.2
38.0 10.8
38.0 11.4
38.0 12.0
38.0 12.6
38.0 13.2
38.0 13.8
38.0 14.4
38.0 15.0
38.0 15.6


KeyboardInterrupt: 